In [1]:
from IPython.display import display
import arcgis
from arcgis.gis import GIS
from arcgis.mapping import WebMap
import datetime
import json

gis = GIS('home')

In [2]:
path = "T:/DCProjects/StoryMap/BikeCounting/BikeCounts/Output/"

In [3]:
# upload items online
for var in ["Hour", "Weekday", "Month", "Season"]:
    file_properties={'title':"Average Bikes Per Hour by " + var,
                 'type': 'Shapefile',
                 'tags': 'bike counts, bike'}
    
    now = datetime.datetime.now()
    file_item = gis.content.add(data=path+"/BPH_"+var+ ".zip",
                            item_properties=file_properties)
    file_item_layer = file_item.publish()
    file_item_layer.share(everyone=True)
    later = datetime.datetime.now()
    elapsed = later - now
    print(elapsed)
    print(var)

0:00:12.255330
Hour
0:00:11.762398
Weekday
0:00:10.928397
Month
0:00:11.245403
Season


In [3]:
avg_bph_agg_map = WebMap()

In [4]:
for var in ["Hour", "Weekday", "Month", "Season"]:
    my_content = gis.content.search(query="owner:" + gis.users.me.username + " AND title:" + "Average Bikes Per Hour by " + var,
                               item_type="Feature Layer")
    file_item=my_content[0]
    avg_bph_agg_map.add_layer(file_item, options={'title':"Average Bikes Per Hour by " + var})

In [5]:
webmap_item_properties = {'title':'Spatial Patterns of Average Bikes Per Hour', 
                         'snippet':'Map created using Python API showing average bikes per hour by hour, weekday, month, and season in central Lane County',
                         'tags':['bike counts', 'bike', 'python']}

In [6]:
avg_bph_agg_map.save(webmap_item_properties)

<Item title:"Spatial Patterns of Average Bikes Per Hour" type:Web Map owner:DCHEN_LCOG>

In [7]:
webmap_search = gis.content.search(query="owner:" + gis.users.me.username + " AND title:" + "Spatial Patterns of Average Bikes Per Hour",
                                  item_type="Web Map")

In [8]:
webmap_item = webmap_search[0]

In [9]:
webmap_item.share(everyone=True)

{'results': [{'itemId': '651a5ee1e56a4ed88f50f3681e592869',
   'success': True,
   'notSharedWith': []}]}

In [22]:
item_data = webmap_item.get_data()

In [11]:
n = len(item_data['operationalLayers'])

In [16]:
def edit_popupInfo(layer):
    field_infos = layer['popupInfo']['fieldInfos']
    # change visibility, lables, and reorder 
    for field in field_infos:
        if field['fieldName'] in ['FID', 'Locatin', 'DblCntL']:
            field["visible"] = False
        if field['fieldName'] == 'BPH':
            field['label'] = 'Bikes Per Hour'
        if field['fieldName'] == 'Latitud':
            field['label'] = 'Latitude'
        if field['fieldName'] == 'Longitd':
            field['label'] = 'Longitude'
        if field['fieldName'] == 'Site_Nm':
            field['label'] = 'Site Name'
        if field['fieldName'] == 'IsOnewy':
            field['label'] = 'Is One Way?'
        if field['fieldName'] == 'OnwyDrc':
            field['label'] = 'One Way Direction'
        if field['fieldName'] == 'IsSdwlk':
            field['label'] = 'Is Sidewalk?'
        if field['fieldName'] == 'N':
            field['label'] = 'Sample Size'
    
    first_item = field_infos.pop(6)
    field_infos.insert(0, first_item)
    
    return layer

In [17]:
for i in range(0, n):
    item_data['operationalLayers'][i] = edit_popupInfo(item_data['operationalLayers'][i])

In [18]:
item_properties = {"text": json.dumps(item_data)}

In [19]:
webmap_item.update(item_properties=item_properties)

True

item_data['operationalLayers']